In [1]:
import pandas as pd
import requests
import json

In [2]:
token = 'e7b7fedd-71d0-48c6-8cc7-749e22ba8e80'

In [3]:
req = requests.get('https://api.sncf.com/v1/coverage/sncf/disruptions//?since=20230201T000000&',auth=(token, ''))

In [4]:
doc = json.loads(req.text)
row = len(doc['disruptions'])
print(f'Nombre de lignes : {row}')

Nombre de lignes : 25


# Retard

## Liste des retards

In [58]:
df = pd.DataFrame(doc['disruptions'])
df.head(5)

,status,disruption_id,severity,impact_id,application_periods,updated_at,uri,impacted_objects,disruption_uri,contributor,cause,id,messages
0,past,43e7b9a8-9622-4bb5-a46c-abaed55ef80d,"{'color': '#000000', 'priority': 42, 'name': '...",43e7b9a8-9622-4bb5-a46c-abaed55ef80d,"[{'begin': '20230227T001000', 'end': '20230227...",20230228T170251,43e7b9a8-9622-4bb5-a46c-abaed55ef80d,[{'impacted_stops': [{'amended_arrival_time': ...,43e7b9a8-9622-4bb5-a46c-abaed55ef80d,realtime.sncf.piv,,43e7b9a8-9622-4bb5-a46c-abaed55ef80d,NaN
1,past,02bfacd7-1e3c-445a-ad1c-c2b48f121965,"{'color': '#000000', 'priority': 42, 'name': '...",02bfacd7-1e3c-445a-ad1c-c2b48f121965,"[{'begin': '20230226T010000', 'end': '20230226...",20230228T170251,02bfacd7-1e3c-445a-ad1c-c2b48f121965,[{'impacted_stops': [{'amended_arrival_time': ...,02bfacd7-1e3c-445a-ad1c-c2b48f121965,realtime.sncf.piv,,02bfacd7-1e3c-445a-ad1c-c2b48f121965,NaN
2,past,a17dec36-dc38-4b7c-998b-0c21622a45f0,"{'color': '#000000', 'priority': 42, 'name': '...",a17dec36-dc38-4b7c-998b-0c21622a45f0,"[{'begin': '20230228T130500', 'end': '20230228...",20230228T170251,a17dec36-dc38-4b7c-998b-0c21622a45f0,[{'impacted_stops': [{'amended_arrival_time': ...,a17dec36-dc38-4b7c-998b-0c21622a45f0,realtime.sncf.piv,,a17dec36-dc38-4b7c-998b-0c21622a45f0,"[{'text': 'Embouteillage', 'channel': {'conten..."
3,past,515a9899-e3d3-47b8-a336-483b2c7cbb89,"{'color': '#000000', 'priority': 42, 'name': '...",515a9899-e3d3-47b8-a336-483b2c7cbb89,"[{'begin': '20230228T062730', 'end': '20230228...",20230228T170251,515a9899-e3d3-47b8-a336-483b2c7cbb89,[{'impacted_stops': [{'amended_arrival_time': ...,515a9899-e3d3-47b8-a336-483b2c7cbb89,realtime.sncf.piv,,515a9899-e3d3-47b8-a336-483b2c7cbb89,"[{'text': 'Travaux sur les voies', 'channel': ..."
4,past,5a5103d0-592d-4315-8999-52f0bb31802d,"{'color': '#000000', 'priority': 42, 'name': '...",5a5103d0-592d-4315-8999-52f0bb31802d,"[{'begin': '20230228T012500', 'end': '20230228...",20230228T170251,5a5103d0-592d-4315-8999-52f0bb31802d,[{'impacted_stops': [{'amended_arrival_time': ...,5a5103d0-592d-4315-8999-52f0bb31802d,realtime.sncf.piv,,5a5103d0-592d-4315-8999-52f0bb31802d,NaN


## Etat du retard

In [140]:
df2 = pd.DataFrame(list(df['severity']))
df2.head(10)

,color,priority,name,effect
0,#000000,42,additional service,ADDITIONAL_SERVICE
1,#000000,42,additional service,ADDITIONAL_SERVICE
2,#000000,42,trip delayed,SIGNIFICANT_DELAYS
3,#000000,42,trip delayed,SIGNIFICANT_DELAYS
4,#000000,42,additional service,ADDITIONAL_SERVICE
5,#000000,42,trip canceled,NO_SERVICE
6,#000000,42,trip delayed,SIGNIFICANT_DELAYS
7,#000000,42,additional service,ADDITIONAL_SERVICE
8,#000000,42,trip delayed,SIGNIFICANT_DELAYS
9,#000000,42,additional service,ADDITIONAL_SERVICE


## Informations complétementaires sur le retard (ligne par ligne)

In [169]:
num_train = 2

df3 = pd.DataFrame(list(df['impacted_objects'][num_train]))
df3 = pd.DataFrame(list(df3['impacted_stops'])[0])
df3 = df3[(df3['amended_arrival_time'].notnull())]

df3 = df3[['stop_point','stop_time_effect','base_arrival_time','amended_arrival_time','cause']]
df3['retard_min'] = df3['amended_arrival_time'].astype('int32') - df3['base_arrival_time'].astype('int32')

def modif_retard(retard):
    return str(retard)[:-2]

df3['retard_min'] = df3['retard_min'].apply(modif_retard)

df3['gare'] = pd.DataFrame(list(df3['stop_point']))['name']
df3 = df3[['gare','stop_time_effect','cause','retard_min']]
df3

,gare,stop_time_effect,cause,retard_min
0,Annemasse,unchanged,,
1,Bons-en-Chablais,delayed,Embouteillage,5
2,Thonon-les-Bains,delayed,Embouteillage,5
3,Évian-les-Bains,delayed,Embouteillage,5
